<a href="https://colab.research.google.com/github/Gabrielalopes15/Treinos-em-python/blob/Minha-vers%C3%A3o-da-imers%C3%A3o/Minha_vers%C3%A3o_de_agentes_de_ia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q --upgrade langchain-google-genai google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.5 MB/s eta 0:00:00


Importação da API

In [2]:
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI

GOOGLE_API_KEY = userdata.get('GEMINI_API_KEY')

Conexão com Gemini
temperature "sentimento"

In [3]:
llm = ChatGoogleGenerativeAI(
    model = "gemini-2.5-flash",
    temperature = 1.0,
    api_key = GOOGLE_API_KEY
)

In [4]:
TRIAGEM_BETHA = (
    "Você é um triador de service desk de um sistema de orgão publico e da os primeiros atendimentos basicos antes de passar pro atendente técnico. "
    "Dada a mensagem do usuário, retorne SOMENTE um JSON com:\n"
    "{\n"
    '  "modulo": "COMPRAS" | "CONTRATOS" | "OBRAS" | "PATRIMONIO" | "ALMOXARIFADO" | "FROTAS",\n'
    '  "submodulo": "PNCP" | "SICOM" | "INTEGRACAO_LICITACAO" | "INTEGRACAO_CONTABIL" | "INTEGRACAO_ALMOXARIFADO" | "INTEGRACAO_PATRIMONIO" | "GERAL",\n'
    '  "prioridade": "CRITICA" | "ALTA" | "MEDIA" | "BAIXA",\n'
    '  "descricao_resumida": "Resumo em até 10 palavras"\n'
    '  "campos adicionais": ["..."]\n'
    "}\n"
    "\n"
    "Regras :\n"
    "- Sempre escolha um MODULO principal de acordo com o contexto da mensagem.\n"
    "  * COMPRAS → licitações, pregão, editais, propostas, PNCP, SICOM. Se o assunto for publicação, PNCP e SICOM é urgente.\n"
    "  * CONTRATOS → termos, aditivos, execução, integração contábil, solicitação, integração com almoxarifado, integração com patrimônio, AM.\n"
    "  * OBRAS → medições, fiscalização, planejamento de obras, SISOP, SICOM.\n"
    "  * PATRIMONIO → bens móveis, tombamento, depreciação, integração com almoxarifado.\n"
    "  * ALMOXARIFADO → estoque, requisições, integrações com patrimônio.\n"
    "  * FROTAS → veículos, abastecimento, motoristas, manutenção, despesas, ordem de abastecimento.\n"
    "\n"
    "- SUBMODULOS:\n"
    "  * PNCP → quando citar 'PNCP' ou 'plataforma nacional de contratações públicas'.\n"
    "  * SICOM → quando mencionar integração com 'TCE', 'SICOM', ou relatórios exigidos.\n"
    "  * INTEGRACAO_LICITACAO → quando falar de integração com portais/licitações externas do tipo BNC, LICITANET, LICITAR DIGITAL, BLL. Essa integração tem prioridade media.\n"
    "  * INTEGRACAO_CONTABIL → quando envolver contabilidade, empenhos, financeiro.\n"
    "  * INTEGRACAO_ALMOXARIFADO → quando falar de estoque integrando com patrimônio.\n"
    "  * INTEGRACAO_PATRIMONIO → quando patrimônio e compras/almoxarifado estiverem relacionados.\n"
    "  * GERAL → se não houver integração ou submódulo específico.\n"
    "- PRIORIDADE:\n"
    "  * CRITICA → sistema totalmente indisponível ou vários usuários afetados.\n"
    "  * ALTA → erros que impedem conclusão de tarefas principais.\n"
    "  * MEDIA → problemas que têm alternativa temporária ou afetam poucos usuários.\n"
    "  * BAIXA → dúvidas de uso, solicitações simples ou informativas.\n"
    "\n"
    "- descricao_resumida: Sempre resuma o problema em até 10 palavras, sem perder o sentido.\n"
    "\n"
)


In [5]:
TELAS_BETHA = {
    "COMPRAS": {
        "PNCP": "Executando > Relatórios > Integração com PNCP ou Apertar a tecla F4 > Integração com PNCP",
        "INTEGRACAO_LICITACAO": "Executando > Relatórios ou Apertar a tecla F4",
    },
    "CONTRATOS": {
        "INTEGRACAO_CONTABIL": "Executando > Solicitação de fornecimento > Interação de dados > + Dados",
        "GERAL": "Menu Principal > Contratos"
    }
}


In [6]:
from pydantic import BaseModel, Field
from typing import Literal, List, Dict

class TriagemOut(BaseModel):
  modulo: Literal ["COMPRAS" , "CONTRATOS" , "OBRAS" , "PATRIMONIO" , "ALMOXARIFADO" , "FROTAS"]
  submodulo: Literal ["PNCP" , "SICOM" , "INTEGRACAO_LICITACAO" , "INTEGRACAO_CONTABIL" , "INTEGRACAO_ALMOXARIFADO" , "INTEGRACAO_PATRIMONIO" , "GERAL"]
  prioridade: Literal ["CRITICA" , "ALTA" , "MEDIA" , "BAIXA"]
  campos_adicionais: List[str] = Field(default_factory = list)


In [7]:
llm_triagem = ChatGoogleGenerativeAI(
    model = "gemini-2.5-flash",
    temperature = 1.0,
    api_key = GOOGLE_API_KEY
)

In [8]:
from langchain_core.messages import SystemMessage, HumanMessage

triagem_chain = llm_triagem.with_structured_output(TriagemOut)

def triagem(mensagem: str) -> Dict:
    saida: TriagemOut = triagem_chain.invoke([
        SystemMessage(content=TRIAGEM_BETHA),
        HumanMessage(content=mensagem)
    ])

    return saida.model_dump()


In [ ]:
testes = ["Como incluir contrato?",
          "preciso enviar um pncp, como faço urgente?",
          "preciso cadastrar um fornecedor como faço?",
          "tem um erro no sicom, sobre o aberlic como faço para corrigir?"]

In [ ]:
for msg_teste in testes:
  print(f"Pergunta: {msg_teste}\n -> Resposta: {triagem(msg_teste)}\n")

Pergunta: Como incluir contrato?
 -> Resposta: {'modulo': 'CONTRATOS', 'submodulo': 'GERAL', 'prioridade': 'BAIXA', 'campos_adicionais': ['Incluir Contrato']}

Pergunta: preciso enviar um pncp, como faço urgente?
 -> Resposta: {'modulo': 'COMPRAS', 'submodulo': 'PNCP', 'prioridade': 'ALTA', 'campos_adicionais': ['Dúvida sobre envio PNCP']}

Pergunta: preciso cadastrar um fornecedor como faço?
 -> Resposta: {'modulo': 'COMPRAS', 'submodulo': 'GERAL', 'prioridade': 'BAIXA', 'campos_adicionais': ['Cadastrar fornecedor no sistema']}

Pergunta: tem um erro no sicom, sobre o aberlic como faço para corrigir?
 -> Resposta: {'modulo': 'COMPRAS', 'submodulo': 'SICOM', 'prioridade': 'ALTA', 'campos_adicionais': ['aberlic']}



palavra chave langchain

Importando os pdf para estudo


In [10]:
!pip install -q --upgrade langchain_community faiss-cpu langchain-text-splitters pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 81.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [25]:
from pathlib import Path
from langchain_community.document_loaders import PyMuPDFLoader

docs = []

for n in Path("/content/").glob("*.pdf"):
  try:
    loader = PyMuPDFLoader(str(n))

    docs.extend(loader.load())
    print(f"Carregado com sucesso arquivo {n.name}")

  except Exception as e:
    print(f"Erro ao carregar arquivo {n.name}: {e}")

print(f"Total de documentos carregados: {len(docs)}")

Carregado com sucesso arquivo L14133.pdf
Carregado com sucesso arquivo COMPRAS E CONTRATOS.pdf
Total de documentos carregados: 73


In [60]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size = 300, chunk_overlap = 30)
chunks = splitter.split_documents(docs)

In [48]:
for chunk in chunks:
  print(chunk)
  print("___________________________")

page_content='Presidência da República
Secretaria-Geral
Subchefia para Assuntos Jurídicos
LEI Nº 14.133, DE 1º DE ABRIL DE 2021
Mensagem de veto
Promulgação partes vetadas
Regulamento
Regulamento
(Vide Decreto nº 12.174, de 2024)
(Vide Decreto nº 12.343, de 2024)    Vigência
Lei de Licitações e Contratos Administrativos.
O  PRESIDENTE  DA  REPÚBLICA Faço saber que o Congresso Nacional decreta e eu' metadata={'producer': 'Skia/PDF m139', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/139.0.0.0 Safari/537.36', 'creationdate': '2025-09-12T10:45:29+00:00', 'source': '/content/L14133.pdf', 'file_path': '/content/L14133.pdf', 'total_pages': 65, 'format': 'PDF 1.4', 'title': 'L14133', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2025-09-12T10:45:29+00:00', 'trapped': '', 'modDate': "D:20250912104529+00'00'", 'creationDate': "D:20250912104529+00'00'", 'page': 0}
___________________________
page_content='sanciono a seguinte Lei:
T

In [61]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(
    model = "models/gemini-embedding-001",
    google_api_key = GOOGLE_API_KEY
)

In [62]:
from langchain_community.vectorstores import FAISS

vectorstores = FAISS.from_documents(chunks, embeddings)

GoogleGenerativeAIError: Error embedding content: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/embed_content_free_tier_requests"
  quota_id: "EmbedContentRequestsPerMinutePerUserPerProjectPerModel-FreeTier"
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
]

In [63]:
retriever = vectorstores.as_retriever(search_type ="similarity_score_threshold",
                                      search_kwargs = {"score_threshold":0.3, "k": 4})

In [64]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt_rag = ChatPromptTemplate.from_messages([
   ("system",
     "Você é um Assistente de leis e informações do sistema Betha da empresa CG CODE "
     "Responda SOMENTE com base no contexto fornecido. "
     "Se não houver base suficiente, responda apenas 'Não sei'."),

    ("human", "Pergunta: {input}\n\nContexto:\n{context}")
])

document_chain = create_stuff_documents_chain(llm_triagem, prompt_rag)




In [65]:
# Formatadores
import re, pathlib

def _clean_text(s: str) -> str:
    return re.sub(r"\s+", " ", s or "").strip()

def extrair_trecho(texto: str, query: str, janela: int = 240) -> str:
    txt = _clean_text(texto)
    termos = [t.lower() for t in re.findall(r"\w+", query or "") if len(t) >= 4]
    pos = -1
    for t in termos:
        pos = txt.lower().find(t)
        if pos != -1: break
    if pos == -1: pos = 0
    ini, fim = max(0, pos - janela//2), min(len(txt), pos + janela//2)
    return txt[ini:fim]

def formatar_citacoes(docs_rel: List, query: str) -> List[Dict]:
    cites, seen = [], set()
    for d in docs_rel:
        src = pathlib.Path(d.metadata.get("source","")).name
        page = int(d.metadata.get("page", 0)) + 1
        key = (src, page)
        if key in seen:
            continue
        seen.add(key)
        cites.append({"documento": src, "pagina": page, "trecho": extrair_trecho(d.page_content, query)})
    return cites[:3]

In [66]:
def perguntar_politica_RAG(pergunta: str) -> Dict:
    docs_relacionados = retriever.invoke(pergunta)

    if not docs_relacionados:
        return {"answer": "Não sei.",
                "citacoes": [],
                "contexto_encontrado": False}

    answer = document_chain.invoke({"input": pergunta,
                                    "context": docs_relacionados})

    txt = (answer or "").strip()

    if txt.rstrip(".!?") == "Não sei":
        return {"answer": "Não sei.",
                "citacoes": [],
                "contexto_encontrado": False}

    return {"answer": txt,
            "citacoes": formatar_citacoes(docs_relacionados, pergunta),
            "contexto_encontrado": True}

In [67]:
testes = ["Como fazer solicitação de fornecimento?",
          "o que é solicitação de compras?",
          "sobre PNCP?",
          "E sicom de 2024, como faço?",
          "baseado na lei 14133, qual artigo fala sobre pregão eletronico?"]

In [68]:
for msg_teste in testes:
    resposta = perguntar_politica_RAG(msg_teste)
    print(f"PERGUNTA: {msg_teste}")
    print(f"RESPOSTA: {resposta['answer']}")
    if resposta['contexto_encontrado']:
        print("CITAÇÕES:")
        for c in resposta['citacoes']:
            print(f" - Documento: {c['documento']}, Página: {c['pagina']}")
            print(f"   Trecho: {c['trecho']}")
        print("------------------------------------")

PERGUNTA: Como fazer solicitação de fornecimento?
RESPOSTA: Para fazer uma solicitação de fornecimento (SF):

1.  Acesse **Contratando**.
2.  Navegue para **contratações e compras**.
3.  Pesquise o **fornecedor**.
4.  Clique na **folha** (que representa a Solicitação de Fornecimento - SF).
5.  Clique em **+solicitações**.
6.  **Preencha os campos** necessários.
7.  **Salve**.

Se estiver utilizando o carrinho de compras:
1. Selecione os itens no **carrinho de compras**.
2. Salve e depois edite para alterar a quantidade ou valor.
CITAÇÕES:
 - Documento: COMPRAS E CONTRATOS.pdf, Página: 7
   Trecho: SOLICITAÇÕES DE COMPRA Solicitação de Compra: Um setor ou usuário identifica a necessidade de comprar um bem ou serviço e faz a solicitação de 
 - Documento: COMPRAS E CONTRATOS.pdf, Página: 3
   Trecho: Incluir SF depois empenho - ORDINÁRIO – Contratando → contratações e compras → pesquisar fornecedor → clicar na folha (S
------------------------------------
PERGUNTA: o que é solicitação de 